In [4]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:96% !important; }</style>"))

In [13]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers import Flatten, Dense, Conv2D, ReLU, LeakyReLU, Reshape
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Sequential

In [39]:
from keras.datasets import mnist

In [11]:
row = 32
col = 32
planes = 1

img_shape = (row, col , planes)
input_dims = 100

## Generator Network

In [23]:
def build_the_generator(input_dims=input_dims, img_shape=img_shape):
    generator = Sequential([
                          Dense(512, input_dim=input_dims), # Input Layer
                          LeakyReLU(alpha=0.01), # f(x) = a
                          Dense(32 * 32 * 1, activation='tanh'), # Output Layer
                          Reshape(img_shape)
                         ])
    return generator
    
    

In [24]:
generator = build_the_generator()

In [25]:
for i, layers_name in enumerate(generator.layers):
    print(i, layers_name)

0 <keras.layers.core.dense.Dense object at 0x000002DF8B9DA350>
1 <keras.layers.activation.leaky_relu.LeakyReLU object at 0x000002DF8C6BAC90>
2 <keras.layers.core.dense.Dense object at 0x000002DF89383D50>
3 <keras.layers.reshaping.reshape.Reshape object at 0x000002DF89393E90>


## Discriminator

In [26]:
def build_the_discriminator(img_shape=img_shape):
    discriminator = Sequential([
        Flatten(input_shape=img_shape),
        Dense(512),
        LeakyReLU(alpha=0.01),
        Dense(1, activation='sigmoid') # Sigmoid, Since, the discriminator is doing binary classification
    ])
    return discriminator

In [27]:
discriminator=build_the_discriminator()

In [28]:
for i, layers_name in enumerate(discriminator.layers):
    print(i, layers_name)

0 <keras.layers.reshaping.flatten.Flatten object at 0x000002DF8C9144D0>
1 <keras.layers.core.dense.Dense object at 0x000002DF8CC27D90>
2 <keras.layers.activation.leaky_relu.LeakyReLU object at 0x000002DF8C673F50>
3 <keras.layers.core.dense.Dense object at 0x000002DF8CC36C10>


In [29]:
def gan_model(generator=generator, discriminator=discriminator):
    model_gan = Sequential([
        generator,
        discriminator
    ])
    return model_gan

In [30]:
gan_model = gan_model()

#### Model Compilation

In [37]:
discriminator.compile(optimizer=tf.keras.optimizers.Adam(),
                     loss=tf.keras.losses.binary_crossentropy,
                     metrics=['accuracy'])

# Freezing the Model
discriminator.trainable = False

# GAN Compilation
gan_model.compile(loss=tf.keras.losses.binary_crossentropy,
                 optimizer=Adam())

In [38]:
for i , layers in enumerate(gan_model.layers):
    print(f"Layer : {i} - {layers}")
    for j, layers_name in enumerate(layers.layers):
        print(f"\tInner Layer : {j} - {layers_name}")

Layer : 0 - <keras.engine.sequential.Sequential object at 0x000002DF8C542050>
	Inner Layer : 0 - <keras.layers.core.dense.Dense object at 0x000002DF8B9DA350>
	Inner Layer : 1 - <keras.layers.activation.leaky_relu.LeakyReLU object at 0x000002DF8C6BAC90>
	Inner Layer : 2 - <keras.layers.core.dense.Dense object at 0x000002DF89383D50>
	Inner Layer : 3 - <keras.layers.reshaping.reshape.Reshape object at 0x000002DF89393E90>
Layer : 1 - <keras.engine.sequential.Sequential object at 0x000002DF8CC34510>
	Inner Layer : 0 - <keras.layers.reshaping.flatten.Flatten object at 0x000002DF8C9144D0>
	Inner Layer : 1 - <keras.layers.core.dense.Dense object at 0x000002DF8CC27D90>
	Inner Layer : 2 - <keras.layers.activation.leaky_relu.LeakyReLU object at 0x000002DF8C673F50>
	Inner Layer : 3 - <keras.layers.core.dense.Dense object at 0x000002DF8CC36C10>


### Model Training

In [51]:
losses = list()
accuracies = list()
chkps = list()

def train_the_model(epochs , batch_size, checkpoint):
    # Loading the dataset
    (X_train, y_train), (X_val, y_val) = mnist.load_data()
    
    # Rescaling the dataset
    X_train = X_train / X_train.max()
    X_train = tf.expand_dims(X_train, axis=3) # Flatten the dataset 
    
    # Real image labels
    real_labels = np.ones((batch_size, 1))
    # Fake image labels
    fake_labels = np.zeros((batch_size, 1))
    
    for iteration in range(1,epochs + 1):
        # Batch of real images
        idx = np.random.randint(0, X_train.shape[0] , batch_size)
        imgs = X_train[idx]
        
        # Batch of fake images 
        z = np.random.normal(0, 1, (batch_size))
        z = tf.expand_dims(z, axis=1) # Flatten Image Matrix
        gen_imgs = generator.predict(z)
        
        # Train Discriminator
        d_loss_real = discriminator.train_on_batch(imgs, real_labels)
        d_loss_fake = discriminator.train_on_batch(gen_imgs, fake_labels)
        d_loss, accuracy = 0.5 * np.add(d_loss_fake, d_loss_real)
        
        # Batch of fake Images for GAN
        z = np.random.normal(0, 1, (batch_size, 1)) # Flatten Image 
        gen_imgs = generator.predict(z)
        
        # GAN loss
        g_loss = gan_model.train_on_batch(z, real_labels)
        
        if (iteration % checkpoint) == 0:
            losses.append(d_loss, g_loss)
            accuracies.append(100 * accuracy)
            chkps.append(iteration)
            
            print(f"[ Iteration : {iteration}, Loss : {d_loss}, Accuracy : {accuracy * 100}, GanLoss : {g_loss} ]")
            show_sample_images(generator)
        

In [41]:
def show_sample_images(generator,rows=4, cols=4):
    
    z = np.random.normal(0, 1, (rows * cols, input_dims))
    gen_imgs = generator.predict(z)
    
    # Rescaling the image 
    #gen_imgs = 0.5 * gen_imgs + 0.5 
    cnt = 0
    plt.figure(figsize=(16,16))
    for row in range(1, rows + 1):
        for col in range(1, cols + 1):
            plt.subplot(row, col , cnt + 1)
            plt.imshow(gen_imgs[cnt, : , : , 0], cmap='gray')
            plt.axis(False)
            cnt += 1
        
    

In [50]:
iterations = 20000
batch_size = 512
sample_interval = 1000

train_the_model(iterations, batch_size, sample_interval)

TypeError: Only integers, slices (`:`), ellipsis (`...`), tf.newaxis (`None`) and scalar tf.int32/tf.int64 tensors are valid indices, got array([37104, 21402, 49647,  2308,  4888,  7605, 56521, 50931, 12039,
       51679, 52852, 52947, 13242, 59265, 37394, 20444, 29963, 43773,
       53203, 32249, 55178, 20857, 47618, 38034,  3612,  1599, 52535,
       11418, 20046, 26416, 38725,  5723, 29328, 30268, 40452, 44107,
       11972,  9157, 32770, 54650, 30934, 15991, 35429, 53844, 58928,
       26727, 37649, 35869, 51812, 46786, 10323, 42968, 58411, 51459,
       50407, 25724, 42662, 50737, 28103, 52879, 22241, 10453, 51701,
       31756, 51815, 24664,  7825, 13991, 47759, 10382, 16148, 38152,
       17079, 19974, 22742, 55730, 57004, 20155, 50717, 45748,  9361,
       40415, 23702, 36451,  2844, 33579, 46646, 45245, 53963, 48034,
        8494, 51934, 39233, 33712, 31045, 22799, 32480, 48996, 45612,
        8088, 17385, 18977, 13308, 59215, 38002, 44439, 54580,  6233,
        5044, 25752, 1